In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Summarization with Large Documents using Document AI and PaLM APIs

> **NOTE:** This notebook uses the PaLM generative model, which will reach its [discontinuation date in October 2024](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text#model_versions). Please refer to [this updated notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/document_processing.ipynb) for a version which uses the latest Gemini model.

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_with_documentai.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_with_documentai.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/language/use-cases/document-summarization/summarization_with_documentai.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Holt Skinner](https://github.com/holtskinner), [Mona Mona](https://github.com/Mona19) |

## Overview

Text summarization is the process of creating a shorter version of a text document while still preserving the most important information. This can be useful for a variety of purposes, such as quickly skimming a long document, getting the gist of an article, or sharing a summary with others.

Although summarizing a short paragraph is a non-trivial task, there are a few challenges to overcome if you want to summarize a large document, such as a PDF file with multiple pages.

[Document AI](https://cloud.google.com/document-ai) provides a scalable and managed way to extract data from documents using AI. In this notebook, you will use the [Document OCR processor](https://cloud.google.com/document-ai/docs/document-ocr), which is a pre-trained model that will extract text and layout information from document files. Document AI provides an API endpoint to access the models, so developers don't have to build and maintain their own models and serving infrastructure.


### Objective

In this notebook we will show how you can do the following:

1. Extract text from PDF documents using the Document AI OCR processor.
1. Use a MapReduce method to chunk the document text.
1. Use PaLM `text-bison` model to generate summaries of the extracted text.


### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI PaLM APIs offered by Generative AI studio
- Document AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
Learn about [Document AI pricing](https://cloud.google.com/document-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


## Getting Started


### Install Vertex AI SDK & Other dependencies


In [ ]:
%pip install --upgrade google-cloud-aiplatform==1.35.0 google-cloud-documentai==2.20.1 google-cloud-documentai-toolbox==0.11.1a backoff==2.2.1 --user

**Colab only**: Run the following cell to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Enable the APIs in your project


In [ ]:
!gcloud config set project "YOUR_PROJECT_ID"
!gcloud services enable documentai.googleapis.com storage.googleapis.com aiplatform.googleapis.com

## Document AI

The following [limits](https://cloud.google.com/document-ai/quotas) apply for online processing with the Document OCR processor.

| Limit             | Value |
| :---------------- | ----: |
| Maximum file size | 20 MB |
| Maximum pages     |    15 |

For documents that don't meet these limits, you can use [batch processing](https://cloud.google.com/document-ai/docs/send-request#batch-process) to extract the document text. (Not covered in this notebook.)


### Preparing data files

To begin, you will need to download PDFs for the summarizing tasks below.
For this notebook, you will be using Alphabet earnings report PDFs hosted in a public Google Cloud Storage bucket.


In [ ]:
# Copying the files from the GCS bucket to local storage
!gsutil -m cp -r gs://github-repo/documents/docai .

### Create Document AI OCR Processor

A [Document AI processor](https://cloud.google.com/document-ai/docs/overview#dai-processors) is an interface between a document file and a machine learning model that performs document processing actions. They can be used to classify, split, parse, or analyze a document. Each Google Cloud project needs to create its own processor instances.

There are two types of Document AI processors:

- Pre-trained processors: These processors are pre-trained on a large dataset of documents and can be used to perform common document processing tasks, such as Optical Character Recognition (OCR), form parsing, and entity extraction.
- Custom processors: These processors can be trained on your own dataset of documents to perform specific tasks that are not covered by the pre-trained processors.

Refer to [Full processor and detail list](https://cloud.google.com/document-ai/docs/processors-list) for all supported processors.

Processors take a PDF or image file as input and output the data in the [`Document`](https://cloud.google.com/document-ai/docs/reference/rest/v1/Document) format.


### Create a processor

Run this code only once to create the processor. You cannot create multiple processors with the same display name. If you receive an error, change the name of the processor and rerun.


In [ ]:
from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import AlreadyExists
from google.cloud import documentai
from google.cloud.documentai_toolbox.wrappers.document import Document

# TODO(developer): Edit these variables before running the code.
project_id = "YOUR_PROJECT_ID"

# See https://cloud.google.com/document-ai/docs/regions for all options.
location = "us"

# Must be unique per project, e.g.: "My Processor"
processor_display_name = "YOUR_PROCESSOR_DISPLAY_NAME"

# You must set the `api_endpoint` if you use a location other than "us".
client_options = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")


def create_processor(
    project_id: str, location: str, processor_display_name: str
) -> documentai.Processor:
    client = documentai.DocumentProcessorServiceClient(client_options=client_options)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(project_id, location)

    # Create a processor
    return client.create_processor(
        parent=parent,
        processor=documentai.Processor(
            display_name=processor_display_name, type_="OCR_PROCESSOR"
        ),
    )


try:
    processor = create_processor(project_id, location, processor_display_name)
    print(f"Created Processor {processor.name}")
except AlreadyExists as e:
    print(
        f"Processor already exits, change the processor name and rerun this code. {e.message}"
    )

### Process the documents

Process document takes the processor name and file path of the document and extracts the text from the document.


In [ ]:
def process_document(
    processor_name: str,
    file_path: str,
) -> documentai.Document:
    client = documentai.DocumentProcessorServiceClient(client_options=client_options)

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Load Binary Data into Document AI RawDocument Object
    raw_document = documentai.RawDocument(
        content=image_content, mime_type="application/pdf"
    )

    # Configure the process request
    request = documentai.ProcessRequest(name=processor_name, raw_document=raw_document)

    result = client.process_document(request=request)

    return result.document

#### Create data chunks

LLMs produce the best results when summarizing documents if the input data is broken up into small "chunks" before being added to the prompt.

The best chunk size depends on the size of the documents. It is a good idea to experiment with different chunk sizes to see what works best for your specific dataset and application.
For the provided documents, we are using the paragraphs detected by Document AI to separate the chunks.
You should experiment with other values as well and see how it affects your summarization.


In [ ]:
import glob
import os

# If you already have a Document AI Processor in your project, assign the full processor resource name here.
processor_name = processor.name
extracted_data: list[dict] = []

# Loop through each PDF file in the "docai" directory.
for path in glob.glob("docai/*.pdf"):
    # Extract the file name and type from the path.
    file_name, file_type = os.path.splitext(path)

    print(f"Processing {file_name}")

    # Process the document.
    document = process_document(processor_name, file_path=path)

    if not document:
        continue

    # Using Document AI Toolbox to handle post-processing
    wrapped_document = Document.from_documentai_document(document)

    # Split the text into chunks based on paragraphs.
    document_chunks = [
        paragraph.text
        for page in wrapped_document.pages
        for paragraph in page.paragraphs
    ]

    # Can also split into chunks by page or blocks.
    # document_chunks = [page.text for page in wrapped_document.pages]
    # document_chunks = [block.text for page in wrapped_document.pages for block in page.blocks]

    # Loop through each chunk and create a dictionary with metadata and content.
    for chunk_number, chunk_content in enumerate(document_chunks, start=1):
        # Append the chunk information to the extracted_data list.
        extracted_data.append(
            {
                "file_name": file_name,
                "file_type": file_type,
                "chunk_number": chunk_number,
                "content": chunk_content,
            }
        )

## Summarization using the [PaLM](https://ai.google/discover/palm2/) Model

You have just used Document AI to extract text from PDF files.

In the next section, you will summarize the extracted text using the PaLM model with Vertex AI.
In order to summarize the text, You can use MapReduce to chunk the text to fit the prompt size.


### Authenticating your notebook environment

- If you are using **Colab** to run this notebook, run the cell below and continue.
- If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).


In [ ]:
from google.colab import auth
import vertexai

auth.authenticate_user()

PROJECT_ID = "YOUR_PROJECT_ID"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

### Import models


In [ ]:
from __future__ import annotations

import backoff
from google.api_core.exceptions import ResourceExhausted
from vertexai.preview.language_models import TextGenerationModel

generation_model = TextGenerationModel.from_pretrained("text-bison")


# This decorator is used to handle exceptions and apply exponential backoff in case of ResourceExhausted errors.
# It means the function will be retried with increasing time intervals in case of this specific exception.
@backoff.on_exception(backoff.expo, ResourceExhausted, max_time=10)
def text_generation_model_with_backoff(**kwargs):
    """
    :param **kwargs: Keyword arguments for the prediction.
    :return: The generated text.
    """
    # Calls the generation_model's 'predict' method with the provided keyword arguments (**kwargs)
    # and then accesses the 'text' attribute to get the generated text.
    return generation_model.predict(**kwargs).text

## MapReduce

MapReduce is a very effective approach for processing large datasets because it is scalable and efficient. It can be used to process datasets that are too large to be processed on a single machine.

Using this approach we will first split the large data into chunks, then running a prompt on each chunk of text. For summarization tasks, the output from the initial prompt would be a summary of that chunk. Once all the initial outputs have been generated, a different prompt is run to combine them. This can be more effective for large datasets.

This consists of two main steps, map and reduce:

- The map step will split the dataset into chunks and run a prompt on each chunk of text. The output from the prompt is a summary of that chunk.

- The reduce step combines the summaries from all the chunks into a single summary.

Here are some pros and cons of using MapReduce method for summarization:

Pros:

- Can summarize a large document
- Can work well with parallel processing as the processes to summarize pages are independent to each other.

Cons:

- Multiple calls to the model is needed
- As the pages are summarized individually, the context between the pages could be lost.


#### Map step

In this section, you will use the model to summarize each chunk of text individually using the initial prompt template.


In [ ]:
prompt_template = """
    Write a concise summary of the following text delimited by triple backquotes.

    ```{text}```

    CONCISE SUMMARY:
"""

# Create an empty list to store the summaries
initial_summary: list[str] = []

# Iterate over the pages and generate a summary for each page
for individual_chunk in extracted_data:
    # Create a prompt for the model using the extracted text and a prompt template
    prompt = prompt_template.format(text=individual_chunk["content"].strip())

    # Generate a summary using the model and the prompt
    summary = text_generation_model_with_backoff(prompt=prompt, max_output_tokens=1024)

    # Append the summary to the list of summaries
    initial_summary.append(summary)

Take a look at the first few summaries of from the initial Map phase. These are the summaries of each individual chunk of text.


In [ ]:
print("\n\n".join(initial_summary[:5]))

#### Reduce step

Here you will create a reduce function that concatenates the summaries from the inital summarization step (Map step) and use the final prompt template to create a summary of the initial summaries.


In [ ]:
# Concatenate the summaries from the inital step
concat_summary = "\n".join(initial_summary)

# Create a prompt for the model using the concatenated text and a prompt template
prompt = prompt_template.format(text=concat_summary)

# Generate a summary using the model and the prompt
summary = text_generation_model_with_backoff(prompt=prompt, max_output_tokens=34)
print(summary)

# Conclusion

In this notebook, you learned:

1. How to use Document AI to extract text from these PDFs.
2. How to use MapReduce to efficiently process large amounts of text data.
3. How to summarize the text extracted from the PDFs using the PaLM `text-bison` model.


## Clean Up

If you no longer need the Document AI processor, you can delete it using the following code.

Alternatively, you can use the Cloud Console to delete the processor as outlined in [Creating and managing processors > Delete a processor](https://cloud.google.com/document-ai/docs/create-processor#documentai_delete_processor-web).


In [ ]:
def delete_processor(processor_name: str) -> None:
    client = documentai.DocumentProcessorServiceClient(client_options=client_options)

    # Delete a processor
    operation = client.delete_processor(name=processor_name)
    # Print operation details
    print(operation.operation.name)
    # Wait for operation to complete
    operation.result()


delete_processor(delete_processor, processor_name)